# TF Keras to Tensorflow & Transfer Learning

Adapted from: https://github.com/Tony607/Keras_catVSdog_tf_estimator

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import shutil
import tensorflow as tf
assert "1.4" <= tf.__version__, "TensorFlow r1.4 or later is needed"

## Prepare the data

In [ ]:
train_cats_dir = '../data/catsdogs/train/cats'
train_dogs_dir = '../data/catsdogs/train/dogs'
test_cats_dir = '../data/catsdogs/test/cats'
test_dogs_dir = '../data/catsdogs/test/dogs'

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

In [ ]:
def create_full_path(folder):
    return [os.path.join(folder, file_name) for file_name in os.listdir(folder)]

In [ ]:
def unison_shuffled_copies(a, b):
    a = np.array(a)
    b = np.array(b)
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
def create_shuffle_dataset(cats_dir, dogs_dir):
    CAT_LABEL = 0
    DOG_LABEL = 1
    
    cats = create_full_path(cats_dir)
    dogs = create_full_path(dogs_dir)
    
    files = cats + dogs
    labels = [CAT_LABEL] * len(cats) + [DOG_LABEL] * len(dogs)
    return unison_shuffled_copies(files, labels)

In [ ]:
train_files, train_labels = create_shuffle_dataset(train_cats_dir, train_dogs_dir)
test_files, test_labels = create_shuffle_dataset(test_cats_dir, test_dogs_dir)

In [ ]:
print(train_files[:10])
print(train_labels[:10])

We have a list of files in input and a list of labels in output

## Build Keras model
We are leveraging the pre-trained VGG16 model's convolution layers. aka the "convolutional base" of the model. Then we add our own classifier fully connected layers to do binary classification(cat vs dog). 

Note that since we don't want to touch the parameters pre-trained in the "convolutional base", so we set them as not trainable. Want to go deeper how this model works? Check out this great [jupyter notebook](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb) by the creator of Keras.

Use keras from tensorflow  `tensorflow.python.keras`. This is new in tensorflow version 1.4.0

In [ ]:
from tensorflow.python.keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.python.keras.optimizers import RMSprop

## Exercise 1

Complete the keras model so that is thas the correct output for a binary classification.

Make sure to check the shape of the output of `conv_base` and plan your layers accordingly.

```python
model = Sequential()
model.add(conv_base)

... your code here

```

## Exercise 2
Freeze all the layers in conv_base, so that it's not trainable. After this the `model.summary` should look like:

    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    vgg16 (Model)                (None, 4, 4, 512)         14714688  
    _________________________________________________________________
    flatten_1 (Flatten)          (None, 8192)              0         
    _________________________________________________________________
    dense_1 (Dense)              (None, 256)               2097408   
    _________________________________________________________________
    dense_2 (Dense)              (None, 1)                 257       
    =================================================================
    Total params: 16,812,353
    Trainable params: 2,097,665
    Non-trainable params: 14,714,688
    _________________________________________________________________

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=2e-5),
              metrics=['acc'])

## Keras model to TF Estimator

`model_dir` will be our location to store trained tensorflow models. 

In [ ]:
model_dir = os.path.join(os.getcwd(), "../models/catvsdog")
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)
os.makedirs(model_dir, exist_ok=True)
print("model_dir: ",model_dir)

### convert Keras model

In [ ]:
from tensorflow.python.keras.estimator import model_to_estimator

In [ ]:
tf_estimator = model_to_estimator(keras_model=model, model_dir=model_dir)

The input_name is the model's input layer name, we will need it later when building Input function for your estimator. More on that in Input function section below.

In [ ]:
input_name = model.input_names[0]
input_name

## Input function
When we train our model, we'll need a function that reads the input image files/labels and returns the image data and labels. Estimators require that you create a function of the following format:
````
def input_fn():
    ...<code>...
    return ({ 'input_1':[ImagesValues]},
            [ImageTypeLogit])
```
The return value must be a two-element tuple organized as follows: :

- The first element must be a dictionary in which each input feature is a key, and then a list of values for the training batch.
- The second element is a list of labels for the training batch.
### Arguments
- **filenames**, an array of image file names
- **labels=None**, an array of the image labels for the model. Set to None for inference
- **perform_shuffle=False**, useful when training, reads batch_size records, then shuffles (randomizes) their order.
- **repeat_count=1**, useful when training, repeat the input data several times for each epoch
- **batch_size=1**, reads batch_size records at a time

## Exercise 3


Let's complete the function below by adding the following steps at the end:

1. map the `_parse_function` onto the dataset
- perform the shuffle if `perform_shuffle` is `True`
- repeat the dataset if there's more than one epoch
- set the batch size
- set the iterator
- return the next batch of features and labels

```
def imgs_input_fn(filenames, labels=None, perform_shuffle=False, repeat_count=1, batch_size=1):
    
    def _parse_function(filename, label):
        image_string = tf.read_file(filename)
        image = tf.image.decode_image(image_string, channels=3)
        image.set_shape([None, None, None])
        image = tf.image.resize_images(image, [150, 150])
        image = tf.subtract(image, 116.779) # Zero-center by mean pixel
        image.set_shape([150, 150, 3])
        image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
        d = dict(zip([input_name], [image])), label
        return d
    
    if labels is None:
        labels = [0]*len(filenames)
    
    labels=np.array(labels)
    
    # Expand the shape of "labels" if necessory
    if len(labels.shape) == 1:
        labels = np.expand_dims(labels, axis=1)
    
    filenames = tf.constant(filenames)
    labels = tf.constant(labels)
    labels = tf.cast(labels, tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    .... your code here
    
    return batch_features, batch_labels
```

In [ ]:
from tensorflow.python.keras.preprocessing import image

## Look at the input function output
Looks like color channels 'RGB' has changed to 'BGR' and shape resized to (150, 150) correctly for our model. That is the input format the VGG16's "convolutional base" is expecting.

In [ ]:
next_batch = imgs_input_fn(test_files, labels=test_labels, perform_shuffle=True, batch_size=20)
with tf.Session() as sess:
    first_batch = sess.run(next_batch)
x_d = first_batch[0]['vgg16_input']

print(x_d.shape)
img = image.array_to_img(x_d[8])
img

Batch labels

In [ ]:
first_batch[1]

## Training the model
Estimators require an `input_fn` with no arguments, so we create a function with no arguments using lambda. Suggested you should only attempt it if you have access to a GPU, it only takes couple minutes. Stop training after "repeat_count" iterations of train data (epochs)

In [ ]:
tf_estimator.train(
    input_fn=lambda: imgs_input_fn(test_files,
                                   labels=test_labels,
                                   perform_shuffle=True,
                                   repeat_count=5,
                                   batch_size=20))

## Evaluate
Evaluate our model using the examples contained in test_files and test_labels

Return value will contain evaluation_metrics such as: loss & average_loss

In [ ]:
evaluate_results = tf_estimator.evaluate(
    input_fn=lambda: imgs_input_fn(test_files, 
                                   labels=test_labels, 
                                   perform_shuffle=False,
                                   batch_size=1))
print("Evaluation results")
for key in evaluate_results:
    print("   {}, was: {}".format(key, evaluate_results[key]))

## Predict
To predict we can set the `labels` to None because that is what we will be predicting.

Here we only predict the first 10 images in the test_files.

In [ ]:
predict_results = tf_estimator.predict(
    input_fn=lambda: imgs_input_fn(test_files[:10], 
                                   labels=None, 
                                   perform_shuffle=False,
                                   batch_size=10))

In [ ]:
predict_logits = []
for prediction in predict_results:
    predict_logits.append(prediction['dense_2'][0])

### Check the prediction result
The model correctly classified all 10 images.

In [ ]:
predict_is_dog = [logit > 0.5 for logit in predict_logits]
actual_is_dog = [label > 0.5 for label in test_labels[:10]]
print("Predict dog:",predict_is_dog)
print("Actual dog :",actual_is_dog)

### tf.estimator.train_and_evaluate

TensorFlow release 1.4 also introduces the utility function **tf.estimator.train_and_evaluate**, which simplifies training, evaluation, and exporting Estimator models. This function enables distributed execution for training and evaluation, while still supporting local execution.

Notice that the train was build on previous training result when we call the `est_catvsdog.train()`

In [ ]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda: imgs_input_fn(test_files,
                                                                   labels=test_labels,
                                                                   perform_shuffle=True,
                                                                   repeat_count=5,
                                                                   batch_size=20), 
                                    max_steps=500)
eval_spec = tf.estimator.EvalSpec(input_fn=lambda: imgs_input_fn(test_files,
                                                                 labels=test_labels,
                                                                 perform_shuffle=False,
                                                                 batch_size=1))

tf.estimator.train_and_evaluate(tf_estimator, train_spec, eval_spec)

## Exercise 4

Read about estimators and datasets in the Tensorflow Programmer's guide:

- https://www.tensorflow.org/programmers_guide/estimators
- https://www.tensorflow.org/programmers_guide/datasets


*Copyright &copy; 2017 Francesco Mosconi & CATALIT LLC. All rights reserved.*